In [ ]:
%matplotlib inline
import jupyanno as ja
from jupyanno.sheets import get_task_sheet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import seaborn as sns
from IPython.display import FileLink, HTML
ja.setup_appmode()
USERNAME = ja.get_user_id()

In [ ]:
task_data = ja.read_task_file('../task.json')
annot_df = ja.read_annotation(get_task_sheet(task_data))
print('Showing most recent 3 annotations')
annot_df.tail(3)

In [ ]:
annot_df = get_task_sheet(task_data)
# anonymize before splitting
anon_ids = {k: 'Anonymous_User_{}'.format(i) for i,k in enumerate(np.unique(annot_df['annotator'].values))}
annot_df['annotator'] = annot_df.apply(lambda c_row: USERNAME 
                                     if c_row['annotator']==USERNAME else 
                                     anon_ids.get(c_row['annotator']),axis=1)
annot_df = ja.read_annotation(annot_df)
print('Showing most recent 3 annotations')
annot_df.tail(3)

In [ ]:
annot_df.groupby(['annotator_class', 'annotator_name']).\
    agg({'viewing_time': ['sum', 'mean'], 'label': len}).\
    reset_index().\
    rename({'label': 'count'}, axis=1).\
    round(2).\
    sort_values(('count', 'len'), ascending=False).\
    style.\
    bar(color='#d65f5f')

In [ ]:
results_list = []
# we need to combine the results from binary class and multiclass problems distinctly

for c_mode, group_annot_df in annot_df.groupby('annotation_mode'):
    group_annot_df = group_annot_df.copy()
    group_annot_df['answer'] = group_annot_df['label']
    if c_mode == 'BinaryClass':
        # make the binary result look like a more standard question
        group_annot_df['label'] = group_annot_df.apply(
            lambda c_row: c_row['task'] if c_row['label'] == 'Yes' else None, 1)
    elif c_mode == 'MultiClass':
        pass
    else:
        print('Dashboard does not support {} problems yet!'.format(c_mode))

    c_results_df = pd.merge(group_annot_df,
                            task_data.data_df,
                            how='left',
                            left_on='item_id',
                            right_on=task_data.image_key_col)

    if c_mode == 'BinaryClass':
        c_results_df['correct'] = c_results_df.apply(lambda x: ja.binary_correct(x, task_data.label_col), 1)
    elif c_mode == 'MultiClass':
        c_results_df['correct'] = c_results_df.apply(
            lambda c_row: c_row['label'] == c_row[task_data.label_col], 1)

    results_list += [c_results_df]
results_df = pd.concat(results_list)

# My Personal Results
Here we show the questions you directly answered

In [ ]:
my_answers = results_df[results_df['annotator'] == USERNAME]['correct'].values
ja.show_my_result([USERNAME], [my_answers.sum()], my_answers.shape[0])

In [ ]:
my_answers_df = results_df[results_df['annotator'] == USERNAME][[
    'Timestamp', 'correct', task_data.image_key_col, 'task', 'answer', task_data.label_col]].copy()

my_answers_df = my_answers_df.sort_values(
    ['Timestamp'], ascending=True).drop(['Timestamp'], 1)
my_answers_df[task_data.image_key_col] = my_answers_df[task_data.image_key_col].map(
    lambda x: ja.path_to_img(os.path.join(task_data.base_img_dir, x)))
html_output = ja.raw_html_render(my_answers_df.rename({task_data.image_key_col: 'Image',
                                           'task': 'Question',
                                           'answer': 'Your Answer',
                                           task_data.label_col: 'Real Answer'}, axis=1))
with open('download_full_table.html', 'w') as f:
    f.write(html_output)
    display(FileLink(f.name))

HTML(html_output)

# Overall Results
We can show the overall results by person / type of user

In [ ]:
results_df.groupby(['annotator_class', 'annotator_name']).\
    agg({'viewing_time': 'mean',
         'label': len,
         'correct': lambda x: 100*np.mean(x),
         'answer_negativity': lambda x: 100*np.mean(x)}).\
    reset_index().\
    rename({'label': 'Count',
            'correct': 'Accuracy (%)',
            'viewing_time': 'Average Viewing Time (s)',
            'annotator_class': 'Type of User',
            'annotator_name': 'Name',
            'answer_negativity': 'Negative Responses (%)'}, axis=1).\
    round(1).\
    sort_values('Accuracy (%)', ascending=False).\
    style.\
    background_gradient(cmap='hot', low=.5, high=0).\
    set_properties(**{'font-size': '12pt'})

## Disease to be identified 
Here we show the breakdown based on which condition the patient actually had

In [ ]:
results_df.groupby(task_data.label_col).\
    agg({'viewing_time': 'mean', 'label': len, 'correct': lambda x: 100*np.mean(x)}).\
    reset_index().\
    rename({'label': 'count',
            'correct': 'Accuracy (%)',
            'viewing_time': 'Average Viewing Time (s)'
            }, axis=1).\
    round(1).\
    sort_values('Accuracy (%)', ascending=False).\
    style.\
    background_gradient(cmap='hot', low=.5, high=0).\
    set_properties(**{'font-size': '12pt'})

## Question Asked
Finally we show the breakdown based on the question asked

In [ ]:
results_df.groupby('task').\
    agg({'viewing_time': 'mean', 'label': len, 'correct': lambda x: 100*np.mean(x)}).\
    reset_index().\
    rename({'label': 'count',
            'correct': 'Accuracy (%)',
            'viewing_time': 'Average Viewing Time (s)',
            'task': 'Question Asked'
            }, axis=1).\
    round(1).\
    sort_values('Accuracy (%)', ascending=False).\
    style.\
    background_gradient(cmap='hot', low=.5, high=0).\
    set_properties(**{'font-size': '12pt'})

# Results vs Random Guesses
Show the results vs Random Guesses for each user

In [ ]:
res_df = results_df.groupby(['annotator', 'annotator_name']).\
    agg({'Timestamp': 'count', 'correct': 'sum'}).\
    reset_index().\
    rename({'Timestamp': 'count'}, axis=1)
for count, c_df in res_df.groupby('count'):
    ax1 = ja.show_my_result(c_df['annotator_name'].values.tolist(),
                         c_df['correct'].values.astype(int).tolist(),
                         count)
    ax1.set_title('{} questions answered'.format(count))